In [62]:
import pandas as pd

df = pd.read_csv("../data/Hotel_Reviews.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

Print out the shape of the data frame you have just loaded (the shape is the number of rows and columns)

In [63]:
print(f"Shape: {df.shape}")

Shape: (515738, 17)


Calculate the frequency count for reviewer nationalities:

    How many distinct values are there for the column Reviewer_Nationality and what are they?
    What reviewer nationality is the most common in the dataset (print country and number of reviews)?
    What are the next top 10 most frequently found nationalities, and their frequency count?

In [64]:
nationalities = df["Reviewer_Nationality"]

nat_freq = nationalities.value_counts()
print(f"Number of nationalities:{len(nat_freq)}")

print(nationalities.value_counts())

print(f"Most common nationality is {nat_freq.index[0]} with {nat_freq[0]} reviews")

print(f"Next top 10 are as follows:\n{nat_freq[1:11]}")

Number of nationalities:227
 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
                               ...  
 Cape Verde                        1
 Northern Mariana Islands          1
 Tuvalu                            1
 Guinea                            1
 Palau                             1
Name: Reviewer_Nationality, Length: 227, dtype: int64
Most common nationality is  United Kingdom  with 245246 reviews
Next top 10 are as follows:
 United States of America     35437
 Australia                    21686
 Ireland                      14827
 United Arab Emirates         10235
 Saudi Arabia                  8951
 Netherlands                   8772
 Switzerland                   8678
 Germany                       7941
 Canada                        7894
 France                        7296
Name: Reviewer_Nationality, dtype: int64


What was the most frequently reviewed hotel for each of the top 10 most reviewer nationalities?

In [65]:
# get top 10 nationalities
top_10 = nat_freq[0:10].keys()

# keep only reviews from top 10 nationalities
nat_group = df[df["Reviewer_Nationality"].isin(top_10)]

# group by nationality then for each apply value count on hotel name, then return count and name of hotel with max count
top_hotels = nat_group.groupby("Reviewer_Nationality").apply(lambda x : (x["Hotel_Name"].value_counts().idxmax(), x["Hotel_Name"].value_counts().max()))

# print data
for nat in top_10:
    hotel, count = top_hotels.get(nat)
    print(f"{nat} has most reviews for {hotel}, with count of {count}.")

 United Kingdom  has most reviews for Britannia International Hotel Canary Wharf, with count of 3833.
 United States of America  has most reviews for Hotel Esther a, with count of 423.
 Australia  has most reviews for Park Plaza Westminster Bridge London, with count of 167.
 Ireland  has most reviews for Copthorne Tara Hotel London Kensington, with count of 239.
 United Arab Emirates  has most reviews for Millennium Hotel London Knightsbridge, with count of 129.
 Saudi Arabia  has most reviews for The Cumberland A Guoman Hotel, with count of 142.
 Netherlands  has most reviews for Jaz Amsterdam, with count of 97.
 Switzerland  has most reviews for Hotel Da Vinci, with count of 97.
 Germany  has most reviews for Hotel Da Vinci, with count of 86.
 Canada  has most reviews for St James Court A Taj Hotel London, with count of 61.


How many reviews are there per hotel (frequency count of hotel) in the dataset?

In [125]:
# First create a new dataframe based on the old one, removing the uneeded columns
hotel_freq_df = df.loc[:, ["Hotel_Name", "Total_Number_of_Reviews"]]

# Group the rows by Hotel_Name, count them and put the result in a new column Total_Reviews_Found
hotel_freq_df["Total_Reviews"] = hotel_freq_df.groupby("Hotel_Name").transform("count")

# Get rid of all the duplicated rows
hotel_freq_df = hotel_freq_df.drop_duplicates(subset = ["Hotel_Name"]).sort_values("Total_Reviews", ascending=False)
display(hotel_freq_df) 

,Hotel_Name,Total_Number_of_Reviews,Total_Reviews
63942,Britannia International Hotel Canary Wharf,9086,4789
164259,Strand Palace Hotel,9568,4256
504027,Park Plaza Westminster Bridge London,12158,4169
440985,Copthorne Tara Hotel London Kensington,7105,3578
236055,DoubleTree by Hilton Hotel London Tower of London,7491,3212
...,...,...,...
485009,Ibis Styles Milano Palmanova,207,12
324004,Hotel Eitlj rg,131,12
481328,Hotel Wagner,135,10
43688,Mercure Paris Porte d Orleans,110,10


While there is an Average_Score column for each hotel in the dataset, you can also calculate an average score (getting the average of all reviewer scores in the dataset for each hotel). Add a new column to your dataframe with the column header Calc_Average_Score that contains that calculated average.

In [162]:
hotel_avg_df = df.loc[:, ["Hotel_Name", "Average_Score"]].drop_duplicates()

#hotel_avg_df["Calc_Average_Score"] = df.groupby("Hotel_Name", as_index=False).mean().values
hotel_avg_df["Calc_Average_Score"] = df.groupby('Hotel_Name').Reviewer_Score.transform('mean')

display(hotel_avg_df)


,Hotel_Name,Average_Score,Calc_Average_Score
0,Hotel Arena,7.7,7.836296
405,K K Hotel George,8.5,8.602827
971,Apex Temple Court Hotel,9.2,9.298843
2008,The Park Grand London Paddington,7.7,7.711469
3778,Monhotel Lounge SPA,8.4,8.848571
...,...,...,...
511962,Suite Hotel 900 m zur Oper,8.0,8.021185
512401,Hotel Amadeus,8.9,8.929861
512545,The Berkeley,9.1,9.064000
512645,Holiday Inn London Kensington,7.8,7.723591


Calculate and print out how many rows have column Negative_Review values of "No Negative"

Calculate and print out how many rows have column Positive_Review values of "No Positive"


In [173]:
no_neg = df.loc[df.Negative_Review == "No Negative", "Negative_Review"].count()
no_pos = df.loc[df.Positive_Review == "No Positive", "Positive_Review"].count()
no_pos_neg = df.loc[df.Positive_Review == "No Positive" or df.Negative_Review == "No Negative", ["Positive_Review", "Negative_Review"]].count()

print(f"{no_neg} No Negatives, {no_pos} No Positives, {no_pos_neg} both No Negatives and No Positives")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().